# 定义模块和窗口 Defining modules and windows

In [ ]:
from IPython.display import display
import ipywidgets.widgets as widgets
import cv2
import time,sys

# 线程功能操作库 Thread function operation library
import threading
import inspect
import ctypes
import matplotlib.pyplot as plt

sys.path.append('/home/pi/project_demo/lib')
#导入麦克纳姆小车驱动库 Import Mecanum Car Driver Library
from McLumk_Wheel_Sports import *


In [ ]:
def _async_raise(tid, exctype):
    """raises the exception, performs cleanup if needed"""
    tid = ctypes.c_long(tid)
    if not inspect.isclass(exctype):
        exctype = type(exctype)
    res = ctypes.pythonapi.PyThreadState_SetAsyncExc(tid, ctypes.py_object(exctype))
    if res == 0:
        raise ValueError("invalid thread id")
    elif res != 1:
        # """if it returns a number greater than one, you're in trouble,
        # and you should call it again with exc=NULL to revert the effect"""
        ctypes.pythonapi.PyThreadState_SetAsyncExc(tid, None)
        
def stop_thread(thread):
    _async_raise(thread.ident, SystemExit)

In [ ]:
def servo_reset():
    bot.Ctrl_Servo(1,90)
    bot.Ctrl_Servo(2,0)
servo_reset()

# 车体状态显示部分进程 Vehicle status display part of the process

##### 自动驾驶 Autopilot

In [ ]:
#转向PID输出值 Steering PID output value
TurnZ_PID_slider = widgets.FloatSlider(description='TurnZ_PID', min=-100, max=100.0, step=0.01, orientation='Vertical')
# create a horizontal box container to place the sliders next to eachother
slider_container = widgets.HBox([TurnZ_PID_slider])
# display the container in this cell's output
display(slider_container)

In [ ]:
#bgr8转jpeg格式 bgr8 to jpeg format
import enum
import cv2

def bgr8_to_jpeg(value, quality=75):
    return bytes(cv2.imencode('.jpg', value)[1])

In [ ]:
#摄像头组件显示 Camera component display
import cv2
import ipywidgets.widgets as widgets
import threading
import time
import numpy as np
import matplotlib.pyplot as plt
import PID


image_widget = widgets.Image(format='jpg', width=640, height=480)
display(image_widget)

In [ ]:
#俯视图 Top view
image_widget_1 = widgets.Image(format='jpg', width=640, height=480)
display(image_widget_1)

In [ ]:
#归一化图+左线（淡绿色） 图像中线（紫色）实际中线（蓝色） 右线（绿色）
#Normalized image + left line (light green) image center line (purple) actual center line (blue) right line (green)
image_widget_2 = widgets.Image(format='jpg', width=640, height=480)
display(image_widget_2)

In [ ]:
image = cv2.VideoCapture(0)
image.set(3,640)
image.set(4,480)
image.set(5, 30)  #设置帧率 Setting the frame rate
# fourcc = cv2.VideoWriter_fourcc(*"MPEG")
# image.set(cv2.CAP_PROP_FOURCC, cv2.VideoWriter.fourcc('M', 'J', 'P', 'G'))
# image.set(cv2.CAP_PROP_BRIGHTNESS, 60) #设置亮度 -64 - 64  0.0  40 Set Brightness -64 - 64 0.0 40
#image.set(cv2.CAP_PROP_CONTRAST, 50) #设置对比度 -64 - 64  2.0  50 Set Contrast -64 - 64 2.0 50
#image.set(cv2.CAP_PROP_EXPOSURE, 156) #设置曝光值 1.0 - 5000  156.0 156 Set exposure value 1.0 - 5000 156.0 156
#ret, frame = image.read()


# from picamera2 import Picamera2, Preview
# import libcamera
# picam2 = Picamera2()  
# camera_config = picam2.create_preview_configuration(main={"format":'RGB888',"size":(640,480)})
# camera_config["transform"] = libcamera.Transform(hflip=1, vflip=1)
# picam2.configure(camera_config) 
# picam2.start() 

#image_widget.value = bgr8_to_jpeg(frame)

# 以下正式开始自动驾驶的进程 The following is the official start of the autonomous driving process

In [ ]:
global Z_axis_pid
Z_axis_pid = PID.PositionalPID(0.5, 0, 1) 
global prev_left
prev_left = 0
global prev_right
prev_right = 0
global flag
flag=1

In [ ]:
def Camera_display():
    global peaks_count
    global prev_left, prev_right,flag
    t_start = time.time()
    fps = 0
    global Z_axis_pid
    # 打开超声波测距功能 Turn on the ultrasonic ranging function
    bot.Ctrl_Ulatist_Switch(1)
    while 1:
        #1、获取图像 Get the image
        #frame = picam2.capture_array()
        ret, frame = image.read()
        
        #帧率显示 Frame rate display
        fps = fps + 1
        mfps = fps / (time.time() - t_start)
        cv2.putText(frame, "FPS: " + str(int(mfps)), (80,80), cv2.FONT_HERSHEY_SIMPLEX, 3.0, (0,255,255), 8)
        
        #图像resize Image resize
        frame = cv2.resize(frame,(320,240))
        
        
        #2、透视变换 (查看俯视图) Perspective Transformation (See Top View)
        matSrc = np.float32([[0, 149],  [320, 149], [281, 72], [43, 72]])
        matDst = np.float32([[0,240], [320,240], [320,0], [0,0]])
        matAffine = cv2.getPerspectiveTransform(matSrc,matDst)# mat 1 src 2 dst
        dst = cv2.warpPerspective(frame,matAffine,(320,240))
        
        #画蓝色的矩形框，采集数据的时候采集的就是当前蓝色矩形框中的数据
        #Draw a blue rectangle. When collecting data, the data collected is the data in the current blue rectangle.
        #src 4->dst 4 (左下角 右下角 右上角 左上角 )
        #src 4->dst 4 (lower left corner, lower right corner, upper right corner, upper left corner)
        pts = np.array([[0, 149],  [320, 149], [281, 72], [43, 72]], np.int32)
        # 顶点个数：4，矩阵变成4*1*2维 
        # OpenCV中需要将多边形的顶点坐标变成顶点数×1×2维的矩阵
        # 这里 reshape 的第一个参数为-1, 表示“任意”，意思是这一维的值是根据后面的维度的计算出来的  
        # Number of vertices: 4, the matrix becomes 4*1*2
        # In OpenCV, the vertex coordinates of the polygon need to be converted into a matrix of the number of vertices × 1 × 2 dimensions
        # Here, the first parameter of reshape is -1, which means "arbitrary", 
        #meaning that the value of this dimension is calculated based on the following dimensions
        pts = pts.reshape((-1, 1, 2))
        cv2.polylines(frame, [pts],True, (255, 0, 0), 3) #True表示闭合，（255,0,0）是蓝色，3是线宽 True means closed, (255,0,0) is blue, and 3 is the line width
        
        #3、转化图像二值化(看二值化之后的黑白图像） Convert the image to binary (see the black and white image after binarization)
        dst_gray = cv2.cvtColor(dst, cv2.COLOR_RGB2GRAY)   #转灰度 Convert to Grayscale
        dst_retval, dst_binaryzation = cv2.threshold(dst_gray, 120, 255, cv2.THRESH_BINARY)   #二值化 Binarization
        dst_binaryzation = cv2.erode(dst_binaryzation, None, iterations=1)                    #腐蚀化操作 Corrosion Operation
        
 
        # 当axis为0时,是压缩行,即将每一列的元素相加,将矩阵压缩为一行
        # 当axis为1时,是压缩列,即将每一行的元素相加,将矩阵压缩为一列(这里的一列是为了方便理解说的，实际上，在控制台的输出中，仍然是以一行的形式输出的)
        # When axis is 0, the row is compressed, that is, the elements of each column are added and the matrix is compressed into one row
        # When axis is 1, it is a compressed column, that is, the elements of each row are added and the matrix is compressed into one column
        histogram = np.sum(dst_binaryzation[dst_binaryzation.shape[0]//2:, :], axis=0)  #我们这里是一维数组
        midpoint = np.int_(histogram.shape[0]/2) #直方图垂直尺寸 Histogram vertical size
        
        left_sum = np.sum(histogram[:20], axis=0)  #计算左边像素点之和 Calculate the sum of the pixels on the left
        right_sum = np.sum(histogram[300:], axis=0)  #计算右边像素点之和 Calculate the sum of the pixels on the right
        
        #print("left_sum =%d "%left_sum)
        #print("right_sum = %d"%right_sum)
        
        
        rightpoint = 320
        center_r = 159
        #print (histogram)
        #print(histogram[::-1])
        #plt.plot(histogram)
        #plt.plot(histogram[::-1])
        #plt.show()
        
        #4、获取黑线左右边线并绘制线条 Get the left and right edges of the black line and draw the line
        #获取左右线的位置 Get the positions of the left and right lines
        leftx_base = np.argmin(histogram[:rightpoint], axis = 0)
        rightx_base = np.argmin(histogram[::-1][:rightpoint], axis = 0) #反转直方图取最右侧的值 Invert the histogram to take the rightmost value
        rightx_base = 319 - rightx_base

        dst_binaryzation = cv2.cvtColor(dst_binaryzation,cv2.COLOR_GRAY2RGB)
        cv2.line(dst_binaryzation,(159,0),(159,240),(255,0,255),2)  #图像中线 紫色  Image center line Purple
        lane_center = int((leftx_base + rightx_base)/2)  #左右线中间位置 The middle position between left and right lines
        #print("lane_center")
        #print(lane_center)
        cv2.line(dst_binaryzation,(leftx_base,0),(leftx_base,240),(0,255,0),2)   # 左侧位置 Left position
        cv2.line(dst_binaryzation,(rightx_base,0),(rightx_base,240),(0,255,0),2) # 右侧位置 Right position
        cv2.line(dst_binaryzation,(lane_center,0),(lane_center,240),(255,0,0),2) # 实际中线位置 Actual midline position
        
        left_sum_value = int(np.sum(histogram[:center_r], axis = 0))/159
        right_sum_value = int(np.sum(histogram[center_r:], axis = 0))/159
        #print("left_sum_value = %d", left_sum_value)
        #print("right_sum_value = %d", right_sum_value)
        #5、计算偏差值 Calculate the deviation value
        Bias = 159 - lane_center
        cv2.putText(dst_binaryzation, "FPS:  " + str(int(mfps)), (10,15), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,255,255), 1)
        cv2.putText(dst_binaryzation, "Bias: " + str(int(Bias)), (10,35), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,255,255), 1)
        #print(Bias)
        
        #转向角PID调节 Steering angle PID adjustment
        Z_axis_pid.SystemOutput = Bias
        Z_axis_pid.SetStepSignal(20)
        Z_axis_pid.SetInertiaTime(0.5, 0.2)
        
        
        if Z_axis_pid.SystemOutput > 20: # 20
            Z_axis_pid.SystemOutput = 20
        elif Z_axis_pid.SystemOutput < -20:
            Z_axis_pid.SystemOutput = -20
            
        TurnZ_PID_slider.value = int(Z_axis_pid.SystemOutput)
        
        speed=15 #不宜过快 should not be too fast
        #cv2.putText(dst_binaryzation, "leftx_base: " + str(int(leftx_base)), (10,55), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,255,255), 1)
        #cv2.putText(dst_binaryzation, "rightx_base: " + str(int(rightx_base)), (10,75), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,255,255), 1)
        # 读取超声波传感器的距离 Reading distance from ultrasonic sensor
        diss_H =bot.read_data_array(0x1b,1)[0]
        diss_L =bot.read_data_array(0x1a,1)[0]
        dis = diss_H << 8 | diss_L
        if(dis <200):
            stop_robot()
            bot.Ctrl_BEEP_Switch(1)  #蜂鸣器开 Buzzer on
            time.sleep(0.05)
            bot.Ctrl_BEEP_Switch(0)  
        else:
            if leftx_base == 0 and rightx_base == 319:
                if prev_left > prev_right:
                    #None
                    rotate_left(speed)
                elif prev_left < prev_right:
                    #None
                    rotate_right(speed)
                    
                prev_left = 0
                prev_right = 0
                
            else: #根据线粗调节Bias值 Adjust the Bias value according to the line thickness
                if Bias > 35:   #黑线在中线的左边，小车左转（左边速度减小，右边速度增大） The black line is to the left of the center line, and the car turns left (the speed on the left decreases, and the speed on the right increases)
                    #prev_left = 1
                    #prev_right = 0
                    if Bias > 140: 
                        rotate_left(int((speed-Z_axis_pid.SystemOutput)/5))
                        prev_left = 0
                        prev_right = 0
                    elif(Bias < 50):
                        move_left(int((speed/15)))
                    else:move_forward(speed)
                        #rotate_left(int((speed/15)))
                    time.sleep(0.001) 
                elif Bias < -35:    #黑线在中线的右边，小车右转   （左边速度增大，右边速度减小）The black line is to the right of the center line, and the car turns right (the speed increases on the left and decreases on the right)
                    #prev_right = 1
                    #prev_left = 0
                    if(Bias < -140):
                        rotate_right(int((speed+Z_axis_pid.SystemOutput)/5))
                        prev_left = 0
                        prev_right = 0
                    elif(Bias > -45):
                        move_right(int((speed/15)))
                    else:move_forward(speed)
                        #rotate_right(int((speed/15)))
                    time.sleep(0.001)
    
                else:
                    #None
                    move_forward(speed)
     
        if left_sum != right_sum:
            if left_sum < right_sum:
                prev_left = prev_left + 1
            elif right_sum < left_sum:
                prev_right = prev_right + 1

        

        image_widget.value = bgr8_to_jpeg(frame)
        image_widget_1.value = bgr8_to_jpeg(dst)
        image_widget_2.value = bgr8_to_jpeg(dst_binaryzation)


In [ ]:
#自动驾驶进程启动 Autonomous driving process starts
thread3 = threading.Thread(target=Camera_display)
thread3.setdaemon=True
thread3.start()

In [ ]:
#结束自动驾驶进程 Ending the Autopilot Process
stop_thread(thread3)
#停止小车运动 Stop the car movement
bot.Ctrl_Ulatist_Switch(0)
bot.Ctrl_BEEP_Switch(0)
stop()

In [ ]:
image.release()
# picam2.stop()
# picam2.close()